In [4]:
# coding=utf-8
"""
This module contains the tests which ensure the the flashloan_tokens parameter is respected when using the b3_two_hop and bancor_v3 arb modes.
"""
from fastlane_bot import Bot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
import subprocess, os, sys
import pytest
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v2.14 (23/May/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.1 (2023-07-03)
UniswapV3 v0.0.1 (2023-07-03)
SushiswapV2 v0.0.1 (2023-07-03)
CarbonV1 v0.0.1 (2023-07-03)
BancorV3 v0.0.1 (2023-07-03)
Version = 3-b2.2 [requirements >= 3.0 is met]


# Setup

In [5]:
def find_main_py():
    # Start at the directory of the current script
    cwd = os.path.abspath(os.path.join(os.getcwd()))
    
    with open("log.txt", "w") as f:
        f.write(f"Searching for main.py in {cwd}")
                
    print(f"Searching for main.py in {cwd}")
    while True:
        # Check if main.py exists in the current directory
        if "main.py" in os.listdir(cwd):
            return cwd  # Found the directory containing main.py
        else:
            # If not, go up one directory
            new_cwd = os.path.dirname(cwd)

            # If we're already at the root directory, stop searching
            if new_cwd == cwd:
                raise FileNotFoundError("Could not find main.py in any parent directory")

            cwd = new_cwd
       
       
def run_command(mode):
    
    # Find the correct path to main.py
    main_script_path = find_main_py()
    print(f"Found main.py in {main_script_path}")
    main_script_path = main_script_path + "/main.py"

    # Run the command
    cmd = [
        "python",
        main_script_path,
        f"--arb_mode={mode}",
        "--default_min_profit_bnt=60",
        "--limit_bancor3_flashloan_tokens=True",
        "--use_cached_events=True",
        "--logging_path=fastlane_bot/data/",
        "--timeout=45"
    ]
    subprocess.Popen(cmd)
        
    # Wait for the expected log line to appear
    expected_log_line = "limiting flashloan_tokens to ["
    found = False
    result = subprocess.run(cmd, text=True, capture_output=True, check=True, timeout=120)

    # Check if the expected log line is in the output
    if expected_log_line in result.stderr:
        found = True

    if not found:
        pytest.fail("Expected log line was not found within 1 minute")  # If we reach this point, the test has failed

## Test Flashloan Tokens b3_two_hop

In [6]:
run_command("b3_two_hop")

Searching for main.py in /Users/mikewcasale/Documents/GitHub/fastlane-bot/resources/NBTest
Found main.py in /Users/mikewcasale/Documents/GitHub/fastlane-bot


2023-07-31 05:04:42,681 [fastlane:INFO] - Using mainnet config
2023-07-31 05:04:42,681 [fastlane:INFO] - Running data fetching for exchanges: ['carbon_v1', 'bancor_v3', 'uniswap_v3', 'uniswap_v2', 'sushiswap_v2']
2023-07-31 05:04:42,948 [fastlane:INFO] - Time taken to add initial pools: 0.09468913078308105
2023-07-31 05:04:43,144 [fastlane:INFO] - Fetching events from 17810870 to 17812870... 0
2023-07-31 05:04:43,145 [fastlane:INFO] - Using cached events
2023-07-31 05:04:43,145 [fastlane:ERROR] - Error in main loop: [Errno 2] No such file or directory: 'fastlane_bot/data/--timeout=45latest_event_data.json'
2023-07-31 05:04:55,392 [fastlane:INFO] - Fetching events from 17810871 to 17812871... 0
2023-07-31 05:04:55,393 [fastlane:INFO] - Using cached events
2023-07-31 05:04:55,393 [fastlane:ERROR] - Error in main loop: [Errno 2] No such file or directory: 'fastlane_bot/data/--timeout=45latest_event_data.json'
2023-07-31 05:05:07,590 [fastlane:INFO] - Fetching events from 17810872 to 17812

KeyboardInterrupt: 